In [21]:
import os
import json
from openai import OpenAI

import data_utils

In [22]:
# import sys
# print(sys.executable)
# !which pip
# !pip show ftfy
# import sys
# !{sys.executable} -m pip install pytorchcv

In [23]:
dataset = "cub"
prompt_type = "around"
model_name = input() #"text-davinci-002"

gpt-4o


In [24]:
client = OpenAI(api_key="")
#open(os.path.join('/Users/arjunvarshney/Documents/DSC190/Label-free-CBM/', ".openai_api_key"), "r").read()[:-1]

In [25]:

prompts = {
    "important" : "List the most important features for recognizing something as a \"goldfish\":\n\n-bright orange color\n-a small, round body\n-a long, flowing tail\n-a small mouth\n-orange fins\n\nList the most important features for recognizing something as a \"beerglass\":\n\n-a tall, cylindrical shape\n-clear or translucent color\n-opening at the top\n-a sturdy base\n-a handle\n\nList the most important features for recognizing something as a \"{}\":",
    "superclass" : "Give superclasses for the word \"tench\":\n\n-fish\n-vertebrate\n-animal\n\nGive superclasses for the word \"beer glass\":\n\n-glass\n-container\n-object\n\nGive superclasses for the word \"{}\":",
    "around" : "List the things most commonly seen around a \"tench\":\n\n- a pond\n-fish\n-a net\n-a rod\n-a reel\n-a hook\n-bait\n\nList the things most commonly seen around a \"beer glass\":\n\n- beer\n-a bar\n-a coaster\n-a napkin\n-a straw\n-a lime\n-a person\n\nList the things most commonly seen around a \"{}\":"
}

base_prompt = prompts[prompt_type]

In [26]:
cls_file = data_utils.LABEL_FILES[dataset]
with open(cls_file, "r") as f:
    classes = f.read().split("\n")

In [27]:
feature_dict = {}

for i, label in enumerate(classes):
    feature_dict[label] = set()
    print("\n", i, label)
    for _ in range(2):
        response = client.responses.create(
              model=model_name,
              input=base_prompt.format(label),
              temperature=0.7,
              #max_tokens=256,
              top_p=1,
              #frequency_penalty=0,
              #presence_penalty=0
            )
        #clean up responses
        features = response.output_text#response.choices[0].text
        features = features.split("\n-")
        features = [feat.replace("\n", "") for feat in features]
        features = [feat.strip() for feat in features]
        features = [feat for feat in features if len(feat)>0]
        features = set(features)
        feature_dict[label].update(features)
    feature_dict[label] = sorted(list(feature_dict[label]))


 0 Black footed Albatross

 1 Laysan Albatross

 2 Sooty Albatross

 3 Groove billed Ani

 4 Crested Auklet

 5 Least Auklet

 6 Parakeet Auklet

 7 Rhinoceros Auklet

 8 Brewer Blackbird

 9 Red winged Blackbird

 10 Rusty Blackbird

 11 Yellow headed Blackbird

 12 Bobolink

 13 Indigo Bunting

 14 Lazuli Bunting

 15 Painted Bunting

 16 Cardinal

 17 Spotted Catbird

 18 Gray Catbird

 19 Yellow breasted Chat

 20 Eastern Towhee

 21 Chuck will Widow

 22 Brandt Cormorant

 23 Red faced Cormorant

 24 Pelagic Cormorant

 25 Bronzed Cowbird

 26 Shiny Cowbird

 27 Brown Creeper

 28 American Crow

 29 Fish Crow

 30 Black billed Cuckoo

 31 Mangrove Cuckoo

 32 Yellow billed Cuckoo

 33 Gray crowned Rosy Finch

 34 Purple Finch

 35 Northern Flicker

 36 Acadian Flycatcher

 37 Great Crested Flycatcher

 38 Least Flycatcher

 39 Olive sided Flycatcher

 40 Scissor tailed Flycatcher

 41 Vermilion Flycatcher

 42 Yellow bellied Flycatcher

 43 Frigatebird

 44 Northern Fulmar

 45 G

In [28]:
json_object = json.dumps(feature_dict, indent=4)
with open("data/concept_sets/gpt4o/gpt4o_{}_{}_new.json".format(dataset, prompt_type), "w") as outfile:
    outfile.write(json_object)